In [ ]:
! pip install textrankr
! pip install spacy

In [2]:
# 라이브러리

from typing import List
from textrankr import TextRank
import requests
import spacy

In [24]:
# 텍스트
text = '''
어느 날 양치기 소년은 마을 사람들에게 소리쳤습니다.
"도와주세요! 늑대예요! 커다란 늑대가 나타났어요!" 
'''

In [22]:
# 동화용 대체 튜플 리스트 ('변경전', '변경후')

replace_word_tuple_list = [('villagers', 'people')]

In [23]:
# papago translate API function
def get_translate(input_text):
    client_id = "rfRvys1WgaW888ughpvm"
    client_secret = "UjTpx8bIkG" 

    data = {'text' : input_text,
            'source' : 'ko',
            'target': 'en'}

    url = "https://openapi.naver.com/v1/papago/n2mt"

    header = {"X-Naver-Client-Id":client_id,
              "X-Naver-Client-Secret":client_secret}

    response = requests.post(url, headers=header, data=data)
    rescode = response.status_code

    if(rescode==200):
        send_data = response.json()
        trans_data = (send_data['message']['result']['translatedText'])
        return trans_data
    else:
        print("Error Code:" , rescode)

# 일반
class MyTokenizer:

  def __call__(self, text: str) -> List[str]:
      tokens: List[str] = text.split()
      return tokens


# summarized function
def summarized(input_text, k=1) :
  mytokenizer: MyTokenizer = MyTokenizer()
  textrank: TextRank = TextRank(mytokenizer)

  summarized: str = textrank.summarize(input_text, k)  

  return summarized


# replace_word
def replace(input_text, replace_word_tuple_list=replace_word_tuple_list):
  
  word_list = input_text.split(" ")  # list화

# [f(x) if condition else g(x) for x in sequence]
  for replace_word in replace_word_tuple_list :

    # 필터링할 단어가 있으면
    if replace_word[0] in word_list :
        word_list = [replace_word[1] if word == replace_word[0] else word for word in word_list] # update

  return " ".join(word_list)


# spacy
def nlp(input_text) :

  # spacy
  _spacy = spacy.load('en')
  doc = _spacy(input_text)

  filtered = []
  for word in doc:
    # word.lemma_ : -PRON- 같은 케이스 예외처리
    if word.lemma_[0] == '-' :
      filtered.append((word, word, word.dep_))
    else : 
      filtered.append((word, word.lemma_, word.dep_))

  # rule기반 문장 추출
  punct_index = -1
  root_index  = 0 

  for idx,i in enumerate(filtered) :

    if i[2] == 'punct' :
      punct_index = idx
    if i[2] == 'ROOT' :
      root_index  = idx

  if   punct_index == -1 :
    return " ".join([str(word[1]) for word in filtered])

  elif punct_index < root_index :
    return " ".join([str(word[1]) for word in filtered[(punct_index+1):]])

  elif punct_index > root_index :
    return " ".join([str(word[1]) for word in filtered[:punct_index]])


# main
def one_shot(input_text) :
  process_1_output = summarized(input_text)
  process_2_output = get_translate(process_1_output)
  process_3_output = replace(process_2_output)
  process_4_output = nlp(process_3_output)

  return process_4_output

In [25]:
one_shot(text)

'the shepherd boy shout at the people'

### 판다스로 전체 추출

In [26]:
text = '''
옛날 어느 마을에 양치기 소년이 살았습니다.
소년은 양들이 매일 풀을 뜯을 수 있도록 양들을 초원으로 데려갔습니다.
"이 근처에는 아무도 없어. 너무 심심해." 

어느 날 양치기 소년은 마을 사람들에게 소리쳤습니다.
"도와주세요! 늑대예요! 커다란 늑대가 나타났어요!" 

깜짝 놀란 마을 사람들은 양치기소년에게 달려갔습니다.
하지만 늑대는 없었습니다.
"무슨 일이야?"
"늑대는 어디 있어?"
"늑대가 나타났다고 외치는 걸 들었어."
"양들은 괜찮아?" 

양치기 소년은 깔깔 웃었습니다.
"하하. 그냥 농담한 거예요!" 

"뭐라고? 깜짝 놀랐잖아!"
"다시는 그러지 마!"
마을 사람들이 화를 냈습니다.
화난 마을 사람은 마을로 돌아갔습니다. 

그 다음날, 양치기소년은 또 다시 장난을 치고 싶었습니다.
양치기 소년은 마을 사람들에게 다시 소리쳤습니다.
"도와주세요! 늑대예요! 커다란 늑대가 나타났어요!" 

깜짝 놀란 마을 사람들이 다시 양치기 소년을 도우러 달려왔습니다.
"무슨 일이야?"
"늑대는 어디 있어?"
"늑대가 나타났다고 외치는 걸 들었어."
"양들은 괜찮아?" 

양치기 소년은 깔깔 웃었습니다.
"와! 이번엔 더 빨리 오셨네요."
"뭐라고? 깜짝 놀랐잖아!"
"다시는 그러지 마!"
마을 사람들은 매우 화를 내며 마을로 돌아갔습니다. 

하지만 어느 날, 늑대가 진짜로 나타났습니다.
놀란 소년은 크게 소리쳤습니다.
"도와주세요! 진짜 늑대예요!" 

하지만 마을 사람들은 들은체도 안했습니다.
"믿을 수 없지. 벌써 세 번째야."
"거짓말임에 틀림없어."
아무도 소년을 도우러 오지 않았습니다.

늑대가 양을 잡아먹기 시작했습니다.
"매에에"
양이 울부짖었습니다.

양치기 소년은 자신의 잘못을 후회했습니다.
하지만 이미 너무 늦어버렸습니다.
'''

In [27]:
import pandas as pd
text_df = pd.DataFrame( text.split('\n\n'), columns = ['text'] )

In [28]:
text_df

,text
0,\n옛날 어느 마을에 양치기 소년이 살았습니다.\n소년은 양들이 매일 풀을 뜯을 수...
1,"어느 날 양치기 소년은 마을 사람들에게 소리쳤습니다.\n""도와주세요! 늑대예요! 커..."
2,깜짝 놀란 마을 사람들은 양치기소년에게 달려갔습니다.\n하지만 늑대는 없었습니다.\...
3,"양치기 소년은 깔깔 웃었습니다.\n""하하. 그냥 농담한 거예요!"""
4,"""뭐라고? 깜짝 놀랐잖아!""\n""다시는 그러지 마!""\n마을 사람들이 화를 냈습니다..."
5,"그 다음날, 양치기소년은 또 다시 장난을 치고 싶었습니다.\n양치기 소년은 마을 사..."
6,"깜짝 놀란 마을 사람들이 다시 양치기 소년을 도우러 달려왔습니다.\n""무슨 일이야?..."
7,"양치기 소년은 깔깔 웃었습니다.\n""와! 이번엔 더 빨리 오셨네요.""\n""뭐라고? ..."
8,"하지만 어느 날, 늑대가 진짜로 나타났습니다.\n놀란 소년은 크게 소리쳤습니다.\n..."
9,"하지만 마을 사람들은 들은체도 안했습니다.\n""믿을 수 없지. 벌써 세 번째야.""\..."


In [29]:
text_df['preprocessing'] = text_df['text'].apply(lambda x : one_shot(x))

In [30]:
text_df

,text,preprocessing
0,\n옛날 어느 마을에 양치기 소년이 살았습니다.\n소년은 양들이 매일 풀을 뜯을 수...,there live a shepherd boy in a village
1,"어느 날 양치기 소년은 마을 사람들에게 소리쳤습니다.\n""도와주세요! 늑대예요! 커...",the shepherd boy shout at the people
2,깜짝 놀란 마을 사람들은 양치기소년에게 달려갔습니다.\n하지만 늑대는 없었습니다.\...,the people run to the shepherd boy
3,"양치기 소년은 깔깔 웃었습니다.\n""하하. 그냥 농담한 거예요!""",the shepherd boy laugh
4,"""뭐라고? 깜짝 놀랐잖아!""\n""다시는 그러지 마!""\n마을 사람들이 화를 냈습니다...",the people be angry
5,"그 다음날, 양치기소년은 또 다시 장난을 치고 싶었습니다.\n양치기 소년은 마을 사...",the shepherd boy want to play a prank again
6,"깜짝 놀란 마을 사람들이 다시 양치기 소년을 도우러 달려왔습니다.\n""무슨 일이야?...",the people run to help the shepherd boy again
7,"양치기 소년은 깔깔 웃었습니다.\n""와! 이번엔 더 빨리 오셨네요.""\n""뭐라고? ...",the shepherd boy laugh
8,"하지만 어느 날, 늑대가 진짜로 나타났습니다.\n놀란 소년은 크게 소리쳤습니다.\n...",a wolf really appear
9,"하지만 마을 사람들은 들은체도 안했습니다.\n""믿을 수 없지. 벌써 세 번째야.""\...",but the people do not even listen


In [33]:
# 1
list(text_df['preprocessing'])

['there live a shepherd boy in a village',
 'the shepherd boy shout at the people',
 'the people run to the shepherd boy',
 'the shepherd boy laugh',
 'the people be angry',
 'the shepherd boy want to play a prank again',
 'the people run to help the shepherd boy again',
 'the shepherd boy laugh',
 'a wolf really appear',
 'but the people do not even listen',
 'the wolf start eat the sheep',
 'the shepherd boy regret his mistake']

In [38]:
print('\n'.join(list(text_df['preprocessing'])))

there live a shepherd boy in a village
the shepherd boy shout at the people
the people run to the shepherd boy
the shepherd boy laugh
the people be angry
the shepherd boy want to play a prank again
the people run to help the shepherd boy again
the shepherd boy laugh
a wolf really appear
but the people do not even listen
the wolf start eat the sheep
the shepherd boy regret his mistake


In [46]:
for idx, i in enumerate(list(text_df['preprocessing'])):
  print((idx*10)*3, i)


0 there live a shepherd boy in a village
30 the shepherd boy shout at the people
60 the people run to the shepherd boy
90 the shepherd boy laugh
120 the people be angry
150 the shepherd boy want to play a prank again
180 the people run to help the shepherd boy again
210 the shepherd boy laugh
240 a wolf really appear
270 but the people do not even listen
300 the wolf start eat the sheep
330 the shepherd boy regret his mistake


In [ ]:
# 2
'''
a shepherd boy lives in town
the shephered boy is shouting at people
people is running and there is a boy
the boy is laughing out loud
There are very angry people
There is a shepherd boy who wants to play a joke
people rush to support the shepherd boy
The shepherd man smiles broadly
there is a wolf
but the people do not listen to what the boy said
the wolf kill the sheep
the shepherd boy regret what he done'''

In [58]:

a = '''a shepherd boy lives in town
the shephered boy is shouting at people
people is running and there is a boy
the boy is laughing out loud
There are very angry people
There is a shepherd boy who wants to play a joke
people rush to support the shepherd boy
The shepherd man smiles broadly
there is a wolf
but the people do not listen to what the boy said
the wolf kill the sheep
the shepherd boy regret what he done'''

In [54]:
a.split('\n')

['a shepherd boy lives in town',
 'the shephered boy is shouting at people',
 'people is running and there is a boy',
 'the boy is laughing out loud',
 'There are very angry people',
 'There is a shepherd boy who wants to play a joke',
 'The shepherd man smiles broadly',
 'there is a wolf',
 'but the people do not listen to what the boy said',
 'the wolf kill the sheep',
 'the shepherd boy regret what he done']

In [59]:
for idx, i in enumerate(a.split('\n')):
  print((idx*10)*3, i)

0 a shepherd boy lives in town
30 the shephered boy is shouting at people
60 people is running and there is a boy
90 the boy is laughing out loud
120 There are very angry people
150 There is a shepherd boy who wants to play a joke
180 people rush to support the shepherd boy
210 The shepherd man smiles broadly
240 there is a wolf
270 but the people do not listen to what the boy said
300 the wolf kill the sheep
330 the shepherd boy regret what he done
